This exercise is adapted from https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/


## Keras Tutorial Overview

There is not a lot of code required, but we are going to step over it slowly so that you will know how to create your own models in the future.

The steps you are going to cover in this tutorial are as follows:

1. Load Data.
2. Define Keras Model.
3. Compile Keras Model.
4. Fit Keras Model.
5. Evaluate Keras Model.
6. Tie It All Together.
7. Make Predictions

This Keras tutorial has a few requirements:

1. You have Python 2 or 3 installed and configured.
2. You have SciPy (including NumPy) installed and configured.
3. You have Keras and a backend (Theano or TensorFlow) installed and configured.

### 1. Load Data
The first step is to define the functions and classes we intend to use in this tutorial.

We will use the NumPy library to load our dataset and we will use two classes from the Keras library to define our model.

The imports required are listed below.

In [2]:
!pip install seaborn

  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)


In [3]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense


import seaborn as sns

sns.set()


Now load the data from *file pima-indians-diabetes.data.csv* stored in Data folder (check Google Drive if unsure)

In [5]:
# load the dataset
dataset = loadtxt('.\Data\pima-indians-diabetes.csv', delimiter=',')

# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

**Q:** Find out more about the dataset - such as how many columns, what kind of information does it provide etc.

In [6]:
import pandas as pd

df = pd.DataFrame(data=dataset)
# df.head()
df.describe()


,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    float64
 1   1       768 non-null    float64
 2   2       768 non-null    float64
 3   3       768 non-null    float64
 4   4       768 non-null    float64
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    float64
 8   8       768 non-null    float64
dtypes: float64(9)
memory usage: 54.1 KB


### 2. Define Keras Model
Models in Keras are defined as a sequence of layers.

We create a **Sequential model** and add layers one at a time until we are happy with our network architecture.

The first thing to get right is to ensure the input layer has the right number of input features. This can be specified when creating the first layer with the **input_dim** argument and setting it to 8 for the 8 input variables.

*Question:* How do we know the number of layers and their types?

This is a very hard question. There are heuristics that we can use and often the best network structure is found through a process of trial and error experimentation (I explain more about this here). Generally, you need a network large enough to capture the structure of the problem.

In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the Dense class. We can specify the number of neurons or nodes in the layer as the first argument, and specify the activation function using the activation argument.

We will use the rectified linear unit activation function referred to as ReLU on the first two layers and the Sigmoid function in the output layer.

It used to be the case that Sigmoid and Tanh activation functions were preferred for all layers. These days, better performance is achieved using the ReLU activation function. We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

We can piece it all together by adding each layer:

* The model expects rows of data with 8 variables (the input_dim=8 argument)
* The first hidden layer has 12 nodes and uses the relu activation function.
* The second hidden layer has 8 nodes and uses the relu activation function.
* The output layer has one node and uses the sigmoid activation function.

In [8]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### 3. Compile Keras Model
Now that the model is defined, we can compile it.

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to map inputs to outputs in our dataset.

We must specify the loss function to use to evaluate a set of weights, the optimizer is used to search through different weights for the network and any optional metrics we would like to collect and report during training.

In this case, we will use cross entropy as the **loss** argument. This loss is for a binary classification problems and is defined in Keras as **“binary_crossentropy“**. 

We will define the optimizer as the efficient stochastic gradient descent algorithm “adam“. This is a popular version of gradient descent because it automatically tunes itself and gives good results in a wide range of problems. 

Finally, because it is a classification problem, we will collect and report the classification accuracy, defined via the metrics argument.

In [9]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 4. Fit Keras Model
We have defined our model and compiled it ready for efficient computation.

Now it is time to execute the model on some data.

We can train or fit our model on our loaded data by calling the fit() function on the model.

Training occurs over epochs and each epoch is split into batches.

* **Epoch:** One pass through all of the rows in the training dataset.
* **Batch:** One or more samples considered by the model within an epoch before weights are updated.

One epoch is comprised of one or more batches, based on the chosen batch size and the model is fit for many epochs.

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify using the epochs argument. We must also set the number of dataset rows that are considered before the model weights are updated within each epoch, called the batch size and set using the batch_size argument.

For this problem, we will run for a small number of epochs (150) and use a relatively small batch size of 10.

These configurations can be chosen experimentally by trial and error. We want to train the model enough so that it learns a good (or good enough) mapping of rows of input data to the output classification. The model will always have some error, but the amount of error will level out after some point for a given model configuration. This is called model convergence.

In [10]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
77/77 [==============================] - 0s 985us/step - loss: 7.3915 - accuracy: 0.6172
Epoch 2/150
77/77 [==============================] - 0s 992us/step - loss: 0.9715 - accuracy: 0.5365
Epoch 3/150
77/77 [==============================] - 0s 960us/step - loss: 0.6773 - accuracy: 0.5521
Epoch 4/150
77/77 [==============================] - 0s 987us/step - loss: 0.6528 - accuracy: 0.6693
Epoch 5/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6461 - accuracy: 0.6654
Epoch 6/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6388 - accuracy: 0.6654
Epoch 7/150
77/77 [==============================] - 0s 934us/step - loss: 0.6308 - accuracy: 0.6719
Epoch 8/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6226 - accuracy: 0.6719
Epoch 9/150
77/77 [==============================] - 0s 948us/step - loss: 0.6171 - accuracy: 0.6797
Epoch 10/150
77/77 [==============================] - 0s 921us/step - loss: 0.6179 - accuracy: 0.

77/77 [==============================] - 0s 1ms/step - loss: 0.5640 - accuracy: 0.6862
Epoch 83/150
77/77 [==============================] - 0s 949us/step - loss: 0.5663 - accuracy: 0.6888
Epoch 84/150
77/77 [==============================] - 0s 975us/step - loss: 0.5608 - accuracy: 0.6927
Epoch 85/150
77/77 [==============================] - 0s 961us/step - loss: 0.5687 - accuracy: 0.6849
Epoch 86/150
77/77 [==============================] - 0s 999us/step - loss: 0.5662 - accuracy: 0.6940
Epoch 87/150
77/77 [==============================] - 0s 987us/step - loss: 0.5604 - accuracy: 0.7031
Epoch 88/150
77/77 [==============================] - 0s 948us/step - loss: 0.5639 - accuracy: 0.6953
Epoch 89/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5615 - accuracy: 0.6901
Epoch 90/150
77/77 [==============================] - 0s 944us/step - loss: 0.5600 - accuracy: 0.6953
Epoch 91/150
77/77 [==============================] - 0s 933us/step - loss: 0.5642 - accuracy: 0.68

**Q:** change the value of Epoch - what differences will you expect/see?

In [11]:
# fit the keras model on the dataset
model.fit(X, y, epochs=300, batch_size=30)

Epoch 1/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5421 - accuracy: 0.6953
Epoch 2/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5381 - accuracy: 0.7083
Epoch 3/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5370 - accuracy: 0.7031
Epoch 4/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5373 - accuracy: 0.7031
Epoch 5/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5374 - accuracy: 0.6992
Epoch 6/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5371 - accuracy: 0.7005
Epoch 7/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5385 - accuracy: 0.6914
Epoch 8/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5409 - accuracy: 0.7044
Epoch 9/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5367 - accuracy: 0.7005
Epoch 10/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5386 - accuracy: 0.6966
Epoch 11/

26/26 [==============================] - 0s 1ms/step - loss: 0.5297 - accuracy: 0.7018
Epoch 83/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5308 - accuracy: 0.7057
Epoch 84/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5297 - accuracy: 0.6979
Epoch 85/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5300 - accuracy: 0.7057
Epoch 86/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5260 - accuracy: 0.7044
Epoch 87/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5276 - accuracy: 0.7057
Epoch 88/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5336 - accuracy: 0.7005
Epoch 89/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5351 - accuracy: 0.7044
Epoch 90/300
26/26 [==============================] - 0s 1000us/step - loss: 0.5278 - accuracy: 0.7109
Epoch 91/300
26/26 [==============================] - 0s 990us/step - loss: 0.5309 - accuracy: 0.6979
Epoch 92

Epoch 163/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5155 - accuracy: 0.7135
Epoch 164/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5161 - accuracy: 0.7083
Epoch 165/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5291 - accuracy: 0.6966
Epoch 166/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5170 - accuracy: 0.7083
Epoch 167/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5172 - accuracy: 0.7070
Epoch 168/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5248 - accuracy: 0.7109
Epoch 169/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5144 - accuracy: 0.7083
Epoch 170/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5148 - accuracy: 0.7057
Epoch 171/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5155 - accuracy: 0.7083
Epoch 172/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5303 - accurac

26/26 [==============================] - 0s 1ms/step - loss: 0.5078 - accuracy: 0.7083
Epoch 244/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5085 - accuracy: 0.7083
Epoch 245/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5062 - accuracy: 0.7109
Epoch 246/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5046 - accuracy: 0.7188
Epoch 247/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5036 - accuracy: 0.7174
Epoch 248/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5044 - accuracy: 0.7109
Epoch 249/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5024 - accuracy: 0.7161
Epoch 250/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5008 - accuracy: 0.7109
Epoch 251/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5068 - accuracy: 0.7096
Epoch 252/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5039 - accuracy: 0.7161
Epoc

### 5. Evaluate Keras Model
We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset.

This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

You can evaluate your model on your training dataset using the **evaluate()** function on your model and pass it the same input and output used to train the model.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

The **evaluate()** function will return a list with two values. The first will be the loss of the model on the dataset and the second will be the accuracy of the model on the dataset. We are only interested in reporting the accuracy, so we will ignore the loss value.

In [12]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 870us/step - loss: 0.5023 - accuracy: 0.7188
Accuracy: 71.88


### 7. Make Predictions
We can adapt the above example and use it to generate predictions on the training dataset, pretending it is a new dataset we have not seen before.

Making predictions is as easy as calling the predict() function on the model. We are using a sigmoid activation function on the output layer, so the predictions will be a probability in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

In [13]:
# make probability predictions with the model
predictions = model.predict(X)

# round predictions 
rounded = [round(x[0]) for x in predictions]


**Q:** now redo predictions by using the function predict_classes(). Show the outcome of the first 10 cases (code provided)

In [15]:
import numpy as np


classes_x=np.argmax(predictions,axis=1)


for i in range(10):
    print('%s => %d (expected %d)' % (X[i].tolist(), classes_x[i], y[i]))


[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 0 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 0 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 0 (expected 1)
[5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0] => 0 (expected 0)
[3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.248, 26.0] => 0 (expected 1)
[10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0] => 0 (expected 0)
[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0] => 0 (expected 1)
[8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.232, 54.0] => 0 (expected 1)


## Challenge 1:
Separate the training and test datasets (70% vs 30%) and rerun the evaluation

In [17]:
!pip install sklearn

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [22]:
# fit the keras model on the dataset
model.fit(X, y, epochs=300, batch_size=30)

Epoch 1/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4955 - accuracy: 0.7201
Epoch 2/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5002 - accuracy: 0.7148
Epoch 3/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4954 - accuracy: 0.7122
Epoch 4/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4937 - accuracy: 0.7305
Epoch 5/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5016 - accuracy: 0.7070
Epoch 6/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4979 - accuracy: 0.7188
Epoch 7/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4925 - accuracy: 0.7161
Epoch 8/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5009 - accuracy: 0.7201
Epoch 9/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5072 - accuracy: 0.7135
Epoch 10/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4981 - accuracy: 0.7253
Epoch 11/

Epoch 83/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4830 - accuracy: 0.7266
Epoch 84/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4892 - accuracy: 0.7161
Epoch 85/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4895 - accuracy: 0.7161
Epoch 86/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4917 - accuracy: 0.7135
Epoch 87/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4900 - accuracy: 0.7161
Epoch 88/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4976 - accuracy: 0.7214
Epoch 89/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4909 - accuracy: 0.7253
Epoch 90/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4892 - accuracy: 0.7214
Epoch 91/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4977 - accuracy: 0.7135
Epoch 92/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4930 - accuracy: 0.7174


26/26 [==============================] - 0s 1ms/step - loss: 0.4900 - accuracy: 0.7148
Epoch 165/300
26/26 [==============================] - 0s 1000us/step - loss: 0.4853 - accuracy: 0.7161
Epoch 166/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4758 - accuracy: 0.7253
Epoch 167/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4802 - accuracy: 0.7201
Epoch 168/300
26/26 [==============================] - 0s 1000us/step - loss: 0.4868 - accuracy: 0.7188
Epoch 169/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4786 - accuracy: 0.7214
Epoch 170/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4749 - accuracy: 0.7240
Epoch 171/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4751 - accuracy: 0.7409
Epoch 172/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4791 - accuracy: 0.7370
Epoch 173/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4841 - accuracy: 0.734

Epoch 245/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4840 - accuracy: 0.7331
Epoch 246/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4756 - accuracy: 0.7539
Epoch 247/300
26/26 [==============================] - 0s 1000us/step - loss: 0.4750 - accuracy: 0.7357
Epoch 248/300
26/26 [==============================] - 0s 1000us/step - loss: 0.4729 - accuracy: 0.7461
Epoch 249/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4727 - accuracy: 0.7331
Epoch 250/300
26/26 [==============================] - 0s 1000us/step - loss: 0.4749 - accuracy: 0.7357
Epoch 251/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4796 - accuracy: 0.7266
Epoch 252/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4697 - accuracy: 0.7422
Epoch 253/300
26/26 [==============================] - 0s 960us/step - loss: 0.4761 - accuracy: 0.7461
Epoch 254/300
26/26 [==============================] - 0s 1ms/step - loss: 0.480

In [24]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 1ms/step - loss: 0.4721 - accuracy: 0.7409
Accuracy: 74.09


In [25]:
# make probability predictions with the model
predictions = model.predict(X)

# round predictions 
rounded = [round(x[0]) for x in predictions]


In [26]:
import numpy as np


classes_x=np.argmax(predictions,axis=1)


for i in range(10):
    print('%s => %d (expected %d)' % (X[i].tolist(), classes_x[i], y[i]))


[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 0 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 0 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 0 (expected 1)
[5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0] => 0 (expected 0)
[3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.248, 26.0] => 0 (expected 1)
[10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0] => 0 (expected 0)
[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0] => 0 (expected 1)
[8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.232, 54.0] => 0 (expected 1)


## Challenge 2:
How can you tune the model to improve the accuracy to be higher than what you have obtained?

A: By increasing the number of epochs

In [37]:
# fit the keras model on the dataset
model.fit(X, y, epochs=1500, batch_size=60)

Epoch 1/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4261 - accuracy: 0.7565
Epoch 2/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4184 - accuracy: 0.7617
Epoch 3/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4152 - accuracy: 0.7669
Epoch 4/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4106 - accuracy: 0.7617
Epoch 5/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4132 - accuracy: 0.7604
Epoch 6/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4182 - accuracy: 0.7578
Epoch 7/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4199 - accuracy: 0.7578
Epoch 8/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4141 - accuracy: 0.7656
Epoch 9/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4205 - accuracy: 0.7604
Epoch 10/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4206 - accuracy: 0.7526

13/13 [==============================] - 0s 1ms/step - loss: 0.4199 - accuracy: 0.7630
Epoch 82/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4178 - accuracy: 0.7565
Epoch 83/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4185 - accuracy: 0.7630
Epoch 84/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4400 - accuracy: 0.7565
Epoch 85/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4251 - accuracy: 0.7526
Epoch 86/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4221 - accuracy: 0.7513
Epoch 87/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4206 - accuracy: 0.7487
Epoch 88/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4330 - accuracy: 0.7435
Epoch 89/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4200 - accuracy: 0.7539
Epoch 90/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4192 - acc

13/13 [==============================] - 0s 1ms/step - loss: 0.4271 - accuracy: 0.7500
Epoch 161/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4148 - accuracy: 0.7578
Epoch 162/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4253 - accuracy: 0.7604
Epoch 163/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4316 - accuracy: 0.7565
Epoch 164/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4380 - accuracy: 0.7630
Epoch 165/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4455 - accuracy: 0.7448
Epoch 166/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4520 - accuracy: 0.7500
Epoch 167/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4496 - accuracy: 0.7422
Epoch 168/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4255 - accuracy: 0.7591
Epoch 169/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4196 - accuracy:

13/13 [==============================] - 0s 1ms/step - loss: 0.4153 - accuracy: 0.7604
Epoch 241/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4180 - accuracy: 0.7591
Epoch 242/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4270 - accuracy: 0.7552
Epoch 243/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4341 - accuracy: 0.7708
Epoch 244/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4762 - accuracy: 0.7096
Epoch 245/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4503 - accuracy: 0.7435
Epoch 246/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4525 - accuracy: 0.7578
Epoch 247/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4425 - accuracy: 0.7435
Epoch 248/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4307 - accuracy: 0.7578
Epoch 249/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4220 - accura

13/13 [==============================] - 0s 1ms/step - loss: 0.4295 - accuracy: 0.7539
Epoch 320/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4256 - accuracy: 0.7500
Epoch 321/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4160 - accuracy: 0.7604
Epoch 322/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4261 - accuracy: 0.7604
Epoch 323/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4314 - accuracy: 0.7539
Epoch 324/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4168 - accuracy: 0.7643
Epoch 325/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4124 - accuracy: 0.7617
Epoch 326/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4100 - accuracy: 0.7708
Epoch 327/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4187 - accuracy: 0.7591
Epoch 328/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4243 - accura

...

13/13 [==============================] - 0s 1ms/step - loss: 0.4244 - accuracy: 0.7604
Epoch 1115/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4231 - accuracy: 0.7669
Epoch 1116/1500
13/13 [==============================] - 0s 917us/step - loss: 0.4148 - accuracy: 0.7617
Epoch 1117/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4168 - accuracy: 0.7721
Epoch 1118/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4184 - accuracy: 0.7565
Epoch 1119/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4138 - accuracy: 0.7617
Epoch 1120/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4159 - accuracy: 0.7617
Epoch 1121/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4183 - accuracy: 0.7630
Epoch 1122/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4269 - accuracy: 0.7409
Epoch 1123/1500
13/13 [==============================] - 0s 1000us/step - loss

13/13 [==============================] - 0s 1000us/step - loss: 0.4223 - accuracy: 0.7461
Epoch 1193/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4261 - accuracy: 0.7591
Epoch 1194/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4277 - accuracy: 0.7396
Epoch 1195/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4187 - accuracy: 0.7591
Epoch 1196/1500
13/13 [==============================] - 0s 999us/step - loss: 0.4168 - accuracy: 0.7617
Epoch 1197/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4306 - accuracy: 0.7565
Epoch 1198/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4226 - accuracy: 0.7500
Epoch 1199/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4139 - accuracy: 0.7591
Epoch 1200/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4131 - accuracy: 0.7604
Epoch 1201/1500
13/13 [==============================] - 0s 1ms/step - loss

13/13 [==============================] - 0s 1ms/step - loss: 0.4198 - accuracy: 0.7565
Epoch 1271/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4362 - accuracy: 0.7461
Epoch 1272/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4327 - accuracy: 0.7487
Epoch 1273/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4173 - accuracy: 0.7643
Epoch 1274/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4214 - accuracy: 0.7513
Epoch 1275/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4124 - accuracy: 0.7487
Epoch 1276/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4089 - accuracy: 0.7669
Epoch 1277/1500
13/13 [==============================] - 0s 917us/step - loss: 0.4191 - accuracy: 0.7526
Epoch 1278/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4160 - accuracy: 0.7513
Epoch 1279/1500
13/13 [==============================] - 0s 1000us/step - loss: 0

13/13 [==============================] - 0s 1ms/step - loss: 0.4147 - accuracy: 0.7565
Epoch 1350/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4200 - accuracy: 0.7591
Epoch 1351/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4129 - accuracy: 0.7630
Epoch 1352/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4135 - accuracy: 0.7643
Epoch 1353/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4156 - accuracy: 0.7591
Epoch 1354/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4171 - accuracy: 0.7630
Epoch 1355/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4161 - accuracy: 0.7604
Epoch 1356/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4137 - accuracy: 0.7578
Epoch 1357/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4238 - accuracy: 0.7617
Epoch 1358/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4279 - acc

13/13 [==============================] - 0s 1ms/step - loss: 0.4264 - accuracy: 0.7526
Epoch 1429/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4095 - accuracy: 0.7630
Epoch 1430/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4215 - accuracy: 0.7656
Epoch 1431/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4097 - accuracy: 0.7604
Epoch 1432/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4064 - accuracy: 0.7669
Epoch 1433/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4054 - accuracy: 0.7669
Epoch 1434/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4125 - accuracy: 0.7669
Epoch 1435/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4158 - accuracy: 0.7565
Epoch 1436/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4338 - accuracy: 0.7461
Epoch 1437/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4333 - 

In [38]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 925us/step - loss: 0.4044 - accuracy: 0.7682
Accuracy: 76.82


## Challenge 3:
How can you save the model and use it again for prediction directly without (re-)training?

In [39]:
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

Saved model to disk
